In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from math import pi, sqrt, log, degrees
import numpy as np
import scipy.sparse.linalg as linalg
import matplotlib

from matplotlib import cm

pd.set_option('display.precision',14)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 1000)
pd.set_option('display.width', 1000)

%matplotlib
FONTSIZE=22

Using matplotlib backend: TkAgg


In [2]:
def phi_psi_to_file(path):

    import glob
    import os
    import matplotlib.pyplot as plt
    from math import pi, radians

    macro_states = glob.glob('%s/macro*.gro'%path)
    print 'Found {0} .gro macrostates'.format(len(macro_states))
    
    #macro_states = glob.glob('%s/new_run*.gro'%path)
    #print 'Found {0} .gro microstates'.format(len(macro_states))
    
    pdb = glob.glob('%s/maxstate*.pdb'%path)
    
    f = open('phi_psi.txt','w')
    f.close()
    
    for conf in macro_states:

        os.system('gmx chi -f {0} -s {1} -rama'.format(conf, pdb[0]))
        f=open('ramaPhiPsiALA2.xvg','r')
        lines = f.readlines()
        f.close()
        p1 = float(lines[-1].strip().split(' ')[0].strip())
        p2 = float(lines[-1].strip().split(' ')[-1].strip())
        phi = p1
        psi = p2

        os.remove('ramaPhiPsiALA2.xvg')
        os.remove('chi.log')
        os.remove('order.xvg')
        f = open('phi_psi.txt','a')
        line = str(phi) + ',' + str(psi) + '\n'
        f.write(line)

In [9]:
def pmf_plot(path):
    
    f = open('phi_psi.txt','r')
    lines = f.readlines()
    phi_psi=[]
    
    for val in lines:
        phi_psi.append((float(val.strip().split(',')[0].strip()),float(val.strip().split(',')[1].strip())))
    
    f.close() 

    print 'Len: ', len(phi_psi)
    print 'Len uniq: ', len(set(phi_psi))
    plot(phi_psi,path)

In [84]:
def plot(phi_psi, path):
    plt.figure()
    
    empty_mat = np.zeros((20,20),dtype=float)
    #extent1 = [min(phi), max(phi), min(psi), max(psi)]
    extent2 = [-180, 180,-180, 180]
    
    #phi_linspace = np.linspace(-180,180, num=36)
    #psi_linspace = np.linspace(180,-180, num=36)
    
    #print phi_linspace
    #print psi_linspace
    
    phi_linspace = range(-180,190,18)
    psi_linspace = range(180,-190,-18)

    phi_coords=[]
    psi_coords=[]
    cnt=0
    phi_vals = []
    psi_vals= []
    for cnt in range(len(phi_psi)):
        #print '------------------------------'
        #print phi[cnt]
        #print psi[cnt]
        
        for i in range(0,len(phi_linspace)-1):
            if float(phi_linspace[i]) < float(phi_psi[cnt][0]) < float(phi_linspace[i+1]):
                #print float(phi_linspace[i]), float(phi[cnt]), float(phi_linspace[i+1])
                #print i
                y=i
                break
        
        for j in range(0,len(psi_linspace)-1):
            if float(psi_linspace[j]) > float(phi_psi[cnt][1]) > float(psi_linspace[j+1]):
                #print float(psi_linspace[i]), float(psi[cnt]), float(psi_linspace[i+1])
                #print j
                x=j
                break
        
        #y = min(range(len(phi_linspace)), key=lambda i: abs(phi_linspace[i]-phi[cnt]))
        #x = min(range(len(psi_linspace)), key=lambda i: abs(psi_linspace[i]-psi[cnt]))
                
        #print 'phi: ',y,'psi: ',x
        #print val
        #print cnt
        #print '------------------------------'
        empty_mat[x,y] += 1
        phi_coords.append(x)
        psi_coords.append(y)
        phi_vals.append(float(phi_psi[cnt][0]))
        psi_vals.append(float(phi_psi[cnt][1]))

    #plt.clf()
    ax = plt.imshow(empty_mat, extent=extent2, cmap = cm.get_cmap('jet_r'), interpolation = "gaussian")

    colors = []
    #colorm = cm.get_cmap('jet_r')
    #for i in range(len(phi)):
        #print phi[i], psi[i]
        #c = colorm(empty_mat[phi_coords[i]][psi_coords[i]])
        #print c
        #colors.append(c)
    
    plt.title('Free Energy Histogram - %s'%path, fontsize=FONTSIZE)
    plt.xlabel(r'$\Phi$ Angle [degrees]', fontsize=FONTSIZE)
    plt.ylabel(r'$\Psi$ Angle [degrees]', fontsize=FONTSIZE)
    
    xticks_1 = np.arange(-180,180, 18)
    xticks = np.append(xticks_1,180)
    yticks_1 = np.arange(-180, 180,18)
    yticks = np.append(yticks_1,180)
    plt.xticks(xticks,rotation=90, fontsize=FONTSIZE)
    plt.yticks(yticks, fontsize=FONTSIZE)
    
    cb = plt.colorbar(pad=0.04)
    cb.set_label('Count',size=FONTSIZE)
    for t in cb.ax.get_yticklabels():
         t.set_fontsize(FONTSIZE) 
    #plt.scatter(phi,psi)
    
    plt.grid()
    plt.show()
    
    fig = plt.gcf()
    fig.set_size_inches(16, 16)
    fig.savefig('plots/free_energy_hist_%s.png'%path, dpi=100)
    

In [85]:
import glob

for i in range(1,11):
    
    phi_psi_to_file('iter%s'%i)
    pmf_plot('iter%s'%i)

Found 700 .gro macrostates
Len:  700
Len uniq:  344
Found 1000 .gro macrostates
Len:  1000
Len uniq:  561
Found 1000 .gro macrostates
Len:  1000
Len uniq:  500
Found 800 .gro macrostates
Len:  800
Len uniq:  415
Found 900 .gro macrostates
Len:  900
Len uniq:  427
Found 1000 .gro macrostates
Len:  1000
Len uniq:  524
Found 1000 .gro macrostates
Len:  1000
Len uniq:  522
Found 800 .gro macrostates
Len:  800
Len uniq:  372
Found 900 .gro macrostates
Len:  900
Len uniq:  402
Found 900 .gro macrostates
Len:  900
Len uniq:  406
